In [10]:
from greedy_mcmc_attack import *
from collections import defaultdict
from torch_geometric.utils import subgraph

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [12]:
# dataset_directory = "../Cora"
cora_dataset = Planetoid(root='', name='Pubmed')
data = cora_dataset[0].to(device)
print(data)

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])


In [21]:
print(len(*nx.connected_components(G)))

19717


In [17]:
node_mask

tensor([True, True, True,  ..., True, True, True], device='cuda:0')

In [15]:
data

Data(x=[19717, 500], edge_index=[2, 88648], y=[19717], train_mask=[19717], val_mask=[19717], test_mask=[19717])

In [4]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [5]:
model_save_path = "../models/pubmed_gcn_model.pth"
list_save_path = "../attacks/pubmed_gcn_edges.pth"
# # multi_metattack_edges_list.pth
# list_save_path = "multi_metattack_edges_list.pth"

In [6]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [7]:
# Get initial accuracy
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.784
Initial Loss: 0.5719911456108093


In [8]:
amts = defaultdict(int)

for _ in range(1):
    attacker = Metattack(data, device=device)
    attacker.setup_surrogate(
        model,
        labeled_nodes=data.train_mask,
        unlabeled_nodes=data.test_mask,
        lambda_=0.0,
    )
    attacker.reset()
    attacker.attack(2)

    for edge, itr in attacker._added_edges.items():
        amts[edge] = itr

    for edge, itr in attacker._removed_edges.items():
        amts[edge] = itr

sorted_list = sorted(amts.items(), key=lambda item: item[1], reverse=False)
sorted_keys = [key for key, value in sorted_list]

sorted_keys

Peturbing graph...:   0%|          | 0/88648 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 2.90 GiB (GPU 0; 31.74 GiB total capacity; 23.63 GiB already allocated; 2.67 GiB free; 26.20 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
list_save_path = "../attacks/pubmed_gcn_edges_full_run.pth"
torch.save(edges_to_add, list_save_path)

In [ ]:
# # save model
# model_save_path = "multi_metattack_gcn_model.pth"
# torch.save(model.state_dict(), model_save_path)

# # save edges
# list_save_path = "multi_metattack_edges_list.pth"
# torch.save(edges_to_add, list_save_path)

In [ ]:
# loaded_model_state_dict = torch.load(model_save_path)

# model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)
# model.load_state_dict(loaded_model_state_dict)

# # Load the saved list
# edges_to_add = torch.load(list_save_path)

In [ ]:
# model_save_path = "multi_metattack_gcn_model.pth"
# list_save_path = "multi_metattack_edges_list.pth"

In [ ]:
# model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, data.x.shape[1], cora_dataset.num_classes, [16], device)

In [ ]:
# G, initial_edge_count, ptb_rate, budget = initialize(data, _ptb_rate=0.15)

In [ ]:
# len(edges_to_add)

In [ ]:
# def two_phase_attack(split):
#     diff_threshold = abs(initial_loss/200)
#     first_phase_edges = int(budget * split)
#     second_phase_percent = ptb_rate * (1 - split) * 1/2
#     print(second_phase_percent)
#     accuracies = []
#     G = to_networkx(data, to_undirected=True)
    
#     data_copy = copy.copy(data)
#     i, j = 0, 0 # i - number added, j - spot in list
#     while i < first_phase_edges:
#         u, v = edges_to_add[j]
    
#         G.add_edge(u, v)
    
#         modified_data = from_networkx(G).to(device)
#         modified_data.x = data.x 
#         modified_data.y = data.y 
#         modified_data.train_mask = data.train_mask
#         modified_data.test_mask = data.test_mask
    
#         modified_loss, modified_accuracy = train.test(modified_data)
#         # print(modified_loss)
    
#         if (abs(modified_loss - initial_loss) / max(modified_loss, initial_loss)) <= diff_threshold:
#         # if modified_accuracy == initial_accuracy:
#             # print(modified_accuracy, i)
#             i += 1
#             # accuracies.append(modified_accuracy)
#             accuracies.append(modified_loss)
#         else:
#             # print(i, 'miss!')
#             G.remove_edge(u, v)
            
#         j += 1
    
#     modified_data = from_networkx(G).to(device)
#     modified_data.x = data.x 
#     modified_data.y = data.y 
#     modified_data.train_mask = data.train_mask
#     modified_data.test_mask = data.test_mask
    
#     attacker = Metattack(modified_data, device=device)
#     attacker.setup_surrogate(model,
#                              labeled_nodes=data.train_mask,
#                              unlabeled_nodes=data.test_mask, lambda_=0.)
#     attacker.reset()
#     attacker.attack(second_phase_percent)

#     degs = defaultdict(tuple)
    
#     for k, v in attacker._added_edges.items():
#         degs[v] = (k, True)
        
#     for k, v in attacker._removed_edges.items():
#         degs[v] = (k, False)
    
#     for _, second in degs.items():
#         u, v = second[0]
#         if second[1]:
#             G.add_edge(u, v)
#         else:
#             G.remove_edge(u, v)
    
#         modified_data = from_networkx(G).to(device)
#         modified_data.x = data.x 
#         modified_data.y = data.y 
#         modified_data.train_mask = data.train_mask
#         modified_data.test_mask = data.test_mask
    
#         modified_loss, modified_accuracy = train.test(modified_data)
    
#         # accuracies.append(modified_accuracy)
#         accuracies.append(modified_loss)

#     print(accuracies)
#     return accuracies

In [ ]:
# splits = [0, 0.5, 0.7, 0.9]
# split_dic_acc = defaultdict(list)
# split_dic_loss = defaultdict(list)
# itrs = defaultdict(int)

In [ ]:
# for s in splits:
#     print(s)
#     split_dic_acc[s], split_dic_loss[s], itrs[s] = two_phase_attack_greedy(data, train, model, s, edges_to_add, constant_fn, device, verbose=True)

In [ ]:
# plot_results(split_dic_acc, ptb_rate, "Greedy", "no", "constant", "accuracy")

In [ ]:
# plot_results(split_dic_loss, ptb_rate, "Greedy", "no", "constant", "loss")

In [ ]:
# itrs